# Проект выполнил: *Светлаков Сергей, студент SkillFactory группы DSPR-38 (DSPR-1)*
# Проект: *Ford VS Ferrari*
# Дата начала работы: *04.05.2021*
# Дата сдачи: *06.05.2021*

## Цель работы: создать и обучить нейронную сеть по типу CNN (сверточная нейронная сеть) для решения задачи классификации изображений по классам. В качестве изображений выступают фото машин, а классов - модели автомобилей. Метрика: точность предсказаний. Данная работа выполнялась на основе имеющегося Base-Line решения на платформе SkillFactory.

***
***В данном notebook представлена финальная версия программы с промежуточными пояснениями к прошлым версиям.***
***

***
***Для ускорения расчетов используется видео-карта, представленная площадкой Kaggle.***
***

In [ ]:
#Информация по видео-карте
!nvidia-smi

# 1. Подключение библиотек

In [ ]:
#Подключение библиотек
#Для работы с данными
import numpy as np
import pandas as pd
#Для визуализации
import matplotlib.pyplot as plt
import seaborn as sns
#Для работы с файлами
import pickle
import zipfile
import csv
import sys
import os
#Для работы с изображениями
import PIL
from PIL import ImageOps, ImageFilter
#Для разбиение выборок
from sklearn.model_selection import train_test_split, StratifiedKFold
#Для работы с ML-CV
import tensorflow as tf
import tensorflow.keras.layers as L                                    #Типы слоев
import tensorflow.keras.models as M                                    #Типы моделей
import tensorflow.keras.optimizers as O                                #Типы решателей
from tensorflow.keras.regularizers import l2                           #Регуляризация для Dense
from tensorflow.keras.preprocessing import image                       
from tensorflow.keras.preprocessing.image import ImageDataGenerator    #Генератор (для test)
from tensorflow.keras.callbacks import Callback,\
                                       EarlyStopping,\
                                       LearningRateScheduler,\
                                       ReduceLROnPlateau,\
                                       ModelCheckpoint

***
***Далее устанавливаются и подключаются еще несколько библиотек для fine-tuning и augmentation.***
***

In [ ]:
#Изменение размера и оформления графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline

In [ ]:
print('Версии используемых библиотек:')
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)
print('Keras        :', tf.keras.__version__)

# 2. Настройки для CV-моделей

***
***Заранее вынесем настройки нейронной сети, перед ее созданием и обучением.***
***Несколько слов о том, как выбирались настройки: сначала были взяты параметры с Base-Line решения SkillFactory, затем при обучении и дообучении модели параметры выбирались так, чтобы получить максимальную точность accuracy, при условии предотвращения возникновения ошибки OOM: out of memory - дефицит машинной памяти на используемом вычислительном ресурсе - видеокарте. Исходный BATCH_SIZE (размер батча) был равен 64 и для fine-tuning выбрана модель EfficientNetB6, но уже при разморозке сети на 50% появлялась ошибка OOM. В итоге была выбрана новая модель для transfer learning - EfficientNetB5, а размер батча снижен до 32, чтобы предотвратить OOM. Количество эпох изначально было равно 5, но было повышено до 15. После предварительных расчетов было выявлено, что количество эпох можно снизить до 8-10, так как дальнейшее обучение модели приводит к снижению метрики качества на валидационной выборке.***
***

In [ ]:
#Параметры сети
RANDOM_SEED    = 42   #Воспроизведение результатов
EPOCHS         = 10   #Количество эпох на обучение
BATCH_SIZE     = 16   #BATCH, подаваемый в сеть
LR             = 1e-3 #Скорость обучения
VAL_SPLIT      = 0.15 #Размер валидационной выборки в сравнении с общей
#Параметры изображения
CLASS_NUM      = 10   #Количество классов в задачей
IMG_SIZE       = 224  #Размер изображения, подаваемого в сеть
IMG_CHANNELS   = 3    #Количество каналов: красный, зеленый, синий
input_shape    = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)
#Пути хранения файлов
DATA_PATH = '../input/sf-dl-car-classification/'  #Директория, содержащая данные
PATH = "../working/car/"                          #Директория, содержащая модели

# 3. EDA

***
***Чтобы не перегружать хранилище (max 20GB) каждый раз перед повторным запуском очищаем директорию working.***
***

In [ ]:
#Очистка папки перед началом работы
!rm -r /kaggle/working/*

In [ ]:
#Чтение обучающей выборки
df_train = pd.read_csv(DATA_PATH+"train.csv")
#Чтение для submit на Kaggle
sample_submission = pd.read_csv(DATA_PATH+"sample-submission.csv")
#Осмотр данных
df_train.head()

***
***Видно, что DataFrame состоит из ссылки на картинку и какому классу принадлежит данная прецендент (картинка).***
***

In [ ]:
#Информация о DF
df_train.info()

***
***В DataFrame нет пропущенных или лишних значений.***
***

In [ ]:
#Распределение прецендентов по классам
df_train['Category'].value_counts()

***
***Присутствует небольшой дисбаланс классов, но на первоначальном этапе попробуем не применять OverSampler или UnderSampler.***
***

In [ ]:
#Распаковка картинок - извлечение в PATH
for data_zip in ['train.zip', 'test.zip']:
    with zipfile.ZipFile(DATA_PATH+data_zip,"r") as z:
        z.extractall(PATH)
#Название извлеченных папок 
print('Название извлеченных папок: {}'.format(os.listdir(PATH)))

In [ ]:
#Извлечение случайной картинки
image = PIL.Image.open(PATH+'/train/0/100380.jpg')
#построение графика
imgplot = plt.imshow(image)
#Отображение
plt.show()

***
***Каждый прецендент содержит фото автомобиля. Посмотрим на еще несколько картинок, чтобы понять: все ли автомобили сфотографированы с одного ракурса?***
***

In [ ]:
#Перебор
for name in df_train['Id'].iloc[:9].values:
    #Извлечение случайной картинки
    image = PIL.Image.open(PATH+'/train/0/'+name)
    #построение графика
    imgplot = plt.imshow(image)
    #Отображение
    plt.show()

***
***Ракурс может быть абсолютно случайным. Может быть случайным и размер фотографии. Также где-то присутствует случайным шум, разная контрасность, цветовая гамма и так далее. Если не учесть все эти факторы - это может привести к обучению некачественной модели. Для улучшения качества предсказаний применим аугментацию данных - сгенерируем на основе имеющихся изображений случайные новые, созданные на основе исходных, путем добавления различных эффектов, разворота/поворота/сдвига и так далее. Стоит упомянуть, что изображение цветное, значит каналов будет 3: red, green, blue.***
***

# 4. Аугментация данных

***
***Для аугментации данных используем библиотеку albumentations. А обертку для TensorFlow скачаем по ссылке от SkillFactore.***
***

In [ ]:
#Используем готовую "продвинутую" библиотеку для аугментации данных
#Ссылка SkillFactory
!pip install git+https://github.com/mjkvaak/ImageDataAugmentor

In [ ]:
#Импорт библиотек для аугментации
import albumentations as alb
from ImageDataAugmentor.image_data_augmentor import *

***
***По ссылке ниже изучим какие можно добавить эффекты на изображения и поэкспериментируем, как добавление/удаление их сказывается на качестве модели при ее обучении.***
***

In [ ]:
#Ссылка - https://albumentations.ai/docs/api_reference/augmentations/transforms/
#         #albumentations.augmentations.transforms.GaussianBlur
p = 0.5 #Вероятность изменить изображение
#Создание аугментатора
augment = alb.Compose([
    #Замена каналов в RGB палитре
    alb.ChannelShuffle(p=p),
    #   #Увеличение изображение RGB по статье Крижевского
    #   alb.FancyPCA(p=p),
    #Размытие изображения
    alb.GaussianBlur(p=p),
    #Добавление шума
    alb.GaussNoise(p=p),
    #   #Добавление шума
    #   alb.GlassBlur(p=p),
    #Поворот вокруг вертикальной оси
    alb.HorizontalFlip(p=p),
    #Изменение оттенка и насыщенности
    alb.HueSaturationValue(p=p),
    #   #Размытие в движение
    #   alb.MotionBlur(p=p),
    #Изменение яркости
    alb.RandomBrightness(p=p,limit=(0.2,0.4)),
    #Изменение контраста
    alb.RandomContrast(p=p,limit=(0.1,0.3)),
    #   #Имитирование вспышки
    #   alb.RandomSunFlare(p=p),
    #Сдвиг в RGB палитре
    alb.RGBShift(p=p),
    #Вращение изображения
    alb.ShiftScaleRotate(shift_limit=0.0625,      #Коэффициент изменения сдвига
                         scale_limit=(0.1,0.2),   #Коэффициент изменение масштаба
                         interpolation=1,         #Флаг для вида интерполяции (линейная)
                         border_mode=4,           #Флаг для экстраполяцц (отражение)
                         rotate_limit=20,         #Угол поворота
                         p=0.7),                  #Вероятность
    #Изменение размера на заданный
    alb.Resize(IMG_SIZE, IMG_SIZE)
])

***
***Опытным путем было установлено, что попытка смоделировать солнечную вспышку или размытость фотографии из-за движения фотографирующего не привела ни к чему хорошему.***
***

In [ ]:
#Генератор для обучающей и валидационной выборок
train_gen = ImageDataAugmentor(rescale=1.0/255,                     #Масштабирование
                               augment=augment,                     #Аугментатор
                               seed=RANDOM_SEED,                    #Для воспроизведения
                               validation_split=VAL_SPLIT)          #Размер выборки
#Генератор для тестовой выборки
test_gen = ImageDataGenerator(rescale=1.0/255)
#Создание обучающей выборки
train_datagen = train_gen.\
            flow_from_directory(PATH+'train/',                      #Путь
                                class_mode='categorical',           #Тип данных
                                batch_size=BATCH_SIZE,              #Размер BATCH'а
                                target_size=(IMG_SIZE, IMG_SIZE),   #Размер матрицы
                                shuffle=True,                       #Перемешиваем заранее
                                subset='training')                  #Название
#Создание валидационной выборки
test_datagen = train_gen.\
            flow_from_directory(PATH+'train/',                      #Путь
                                class_mode='categorical',           #Тип данных
                                batch_size=BATCH_SIZE,              #Размер BATCH'а
                                target_size=(IMG_SIZE, IMG_SIZE),   #Размер матрицы
                                shuffle=True,                       #Перемешиваем заранее
                                subset='validation')                #Название
#Создание тестовой выборки
test_sub_generator = test_gen.\
            flow_from_dataframe(dataframe=sample_submission,
                                x_col="Id",
                                y_col=None,
                                directory=PATH+'test_upload/',
                                class_mode=None,
                                batch_size=BATCH_SIZE,
                                shuffle=False,
                                target_size=(IMG_SIZE, IMG_SIZE),
                                seed=RANDOM_SEED)

***
***Создали 2 генератора и на их основе сгенерировали 3 группы изображений: обучающая и валидационная выборки (train_gen) и тестовая выборка (test_gen).***
***

In [ ]:
#Импорт библиотеки для просмотра изображений
from skimage import io

#Функция для просмотра изображений
def imshow(image_RGB):
    io.imshow(image_RGB)
    io.show()
    pass

#Создание массива
x,y = train_datagen.next()
#Размер картинки
plt.figure(figsize=(12,12))
#Перебор
for i in range(0,9):
    #Извлечении картинки
    image = x[i]
    #Создание места для графика
    plt.subplot(3,3, i+1)
    #Добавление картинки
    plt.imshow(image)
#Вывод
plt.show()

***
***Видно, что аугментация прошла успешна. Видны примеры измения контрасности, освещения и т.д.***
***

# 5. Извлечение 'головы' и создание модели - SOTA

***
***Для улучшения качества предсказаний в достаточно ограниченном количестве ресурсов, как временных, так и вычислительных, воспользуемся готовой обученной моделью и постепенно переобучим готовую модель под данную задачу. Данная операция называется fine-tuning, если обучение частичное и transfer learning, если переобучается вся 'голова'.***
***

In [ ]:
#https://paperswithcode.com/sota/image-classification-on-imagenet
#Установка библиотеки
!pip install -q efficientnet

In [ ]:
#Импорт библиотек для аугментации
import efficientnet.tfkeras as efn

***
***Изначально в качестве 'головы' модели выбрана модель EfficientNetB6, но из-за большого количества весов в ней (60 млн.) сеть не поддается обучению из-за ограничения по количеству используемой памяти. Поэтому модель была заменена на EfficientNetB5. Также тестировалась модель Xception, но она показывала результат на 1-2% хуже.***
***

In [ ]:
#Базовая модель
base_model = efn.EfficientNetB5(weights='imagenet',        #Обученная на imagenet
                                include_top=False,         #Включать ли верхнюю часть сети
                                input_shape=input_shape)   #Размер матрицы

***
***Далее необходимо сконструировать архитектуру сети. Ниже представлена финальная версия, которая дала наилучший результат. Также в качестве комментариев под кодом приведены различные ранние версии и причины, по которым они не используются.***
***

In [ ]:
#Создание модели для обучения
model = M.Sequential()
#Добавление слоев
model.add(base_model)                                #Слой 1 - базовая модель B6
model.add(L.GlobalAveragePooling2D())                #Слой 2 - пулинг слой
model.add(L.Dense(256,                               #Слой 3 - обычная сеть
                  activation='relu',
                  bias_regularizer=l2(1e-4),
                  activity_regularizer=l2(1e-5)))
model.add(L.BatchNormalization())                    #Слой 4 - Batch-нормализация
model.add(L.Dropout(0.25))                           #Слой 5 - Drop нейронов
model.add(L.Dense(CLASS_NUM,                         #Слой 6 - Выходной слой
                  activation='softmax'))

***
***В финальной версии для ускорения сходимости применяется Батч-нормализация. Также для предотвращения переобучения применяется регуляризация к Dense слою. Дополнительно для предотвращения переобучения применяется DropOut слой.***
***

In [ ]:
#Вариант 1: Dense cлой 256 + relu (итоговый)
#Вариант 2: Dense cлой 512 + relu
#Вариант 3: Dense cлой 256 + elu
#Вариант 4: Dense cлой 256x256 + relu

***
***Вариант 2 практически не дал никакой разницы, лишь увеличил время обучения. Вариант 3 улучшил качество на валидационной выборке, но ухудшил на Kaggle (скорее всего переобучение). Вариант 4 не также, как и вариант 3 улучшил качество, но совсем незначительно. Для вариант 4 после дополнительного Dense слоя применяется батч-нормализация и drop-out слой.***
***

# 6. Обучение модели (transfer-learning+finetuning)

***
***Для улучшения качества обучения добавим функции, которые помогают фиксировать лучший результат и изменять темп обучения в автоматическом режиме. Была попытка использовать LearningRateScheduler, но она не привела к значимым результатам.***
***

In [ ]:
#Сохранение прогресса обучения
#Сохранение модели
checkpoint = ModelCheckpoint('best_model.hdf5', 
                             monitor = ['val_accuracy'],    #Метрика контроля
                             verbose = 1,                   #Вывод информации
                             mode = 'max')                  #Max metrics
#Ранняя оставка, когда метрика не растет
earlystop = EarlyStopping(monitor = 'val_accuracy',         #Метрика контроля
                          patience = 5,                     #Кол-во эпох без роста
                          restore_best_weights = True)      #Если провал, то возврат?
#Уменьшение темпа обучения, когда метрика падает
reduce_lr = ReduceLROnPlateau(monitor='val_loss',           #Метрика контроля
                              factor=0.2,                   #Во сколько раз снижается
                              patience=3,                   #Кол-во эпох без улучшения
                              min_lr=0.000001,              #Минимальное значение
                              verbose=1,                    #Вывод информации
                              mode='auto')                  #Какая величина контролируется
#Полный список callbacks
callbacks_list = [checkpoint, earlystop, reduce_lr]

***
***Так как в данном notebook повторяется из раза в раз обучение одной и той же модели, то была специальна написана функция для сокращения общего количества кода.***
***

In [ ]:
#Функции для обучения моделей
def get_plot_history(history):
    '''
    Построение графиков: функция потерь и метрика качества на выборках в зав-ти от эпох.
    Вход:
    * history - модель.
    Выход:
    * None.
    '''
    #Вывод метрики
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    #Вывод функции потерь
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    #Количество эпох
    epochs = range(len(acc))
    #Построение графика - метрика качества
    plt.plot(epochs, acc, 'b', label='Training acc')
    plt.plot(epochs, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    #Построение графика - функция потерь
    plt.figure()
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    #Показ графиков
    plt.show()

In [ ]:
def train_model(model,base_model,size,lr,name,key_fit,train_datagen,test_datagen,EPOCHS):
    '''
    Функция для обучения модели с размороженной головой.
    Вход:
    * model - модель для обучения;
    * base_model - голова модели;
    * size - размер заморозки головы модели;
    * lr - темп обучения;
    * name - индекс в названии файла модели;
    * train_datagen - сгенерированная обучающая выборка;
    * test_datagen - сгенерированная валидационная выборка;
    * EPOCHS - число эпох.
    Выход:
    * model, base_model.
    '''
    #Обучение модели
    base_model.trainable = True
    #Количество слоев
    len_lay = len(base_model.layers)
    #Количество слоев для заморозки
    fine_tune_lay = int(len_lay * size)
    #Заморозка
    for layer in base_model.layers[:fine_tune_lay]:
        layer.trainable =  False
    #Компиляция задачи: модель, метрика и функция потерь
    model.compile(loss="categorical_crossentropy", 
                  optimizer=O.Adam(lr=lr), 
                  metrics=["accuracy"])
    #Скелет модели
    model.summary()
    #Обучение
    if key_fit == 1:
        history = model.fit(
            train_datagen,
            steps_per_epoch = len(train_datagen),
            validation_data = test_datagen, 
            validation_steps = len(test_datagen),
            epochs = EPOCHS,
            callbacks = callbacks_list)
        #Сохранение
        model.save('../working/best_model_'+name+'.hdf5')
        #Метрика качества
        scores = model.evaluate(test_datagen, verbose=1)
        print("Accuracy: %.3f%%" % (scores[1]*100))
        #Графики
        get_plot_history(history)
    else:
        model.load_weights('../working/best_model_'+name+'.hdf5')
    return model, base_model

***
***Если модель обучается, то 1; если читается готовая, то 0.***
***

In [ ]:
#Обучать (1) или читать модель (0)
key_fit = 1

***
***Убедимся, что работает именно GPU.***
***

In [ ]:
#Список используемых устройств
tf.config.list_physical_devices('GPU')

***
***Далее постепенно размораживаем готовую модель и дообучаем всю модель на 10 эпохах. При размораживании 'головы' уменьшаем темп обучения в 5-10 раз. Количество эпох подобрано опытным путем. Чтобы не получать снижение качества на валидации необходимо выбирать количество эпох от 8 до 10. Были попытки увеличивать темп обучения после его снижения (чтобы выскочить из предполагаемого локального оптимума) и в дальнейшем снижать его, но это также не дало значимых результатов.***
***

In [ ]:
#Исходная модель: голова заморожена, обучаем хвост
model, base_model = train_model(model,base_model,1.00,LR*1.00,'0',
                                key_fit,train_datagen,test_datagen,EPOCHS)

In [ ]:
#Первая модель: голова на 50% разморожена, обучаем хвост и оставшуюся часть головы
model, base_model = train_model(model,base_model,0.50,LR*0.10,'1',
                                key_fit,train_datagen,test_datagen,EPOCHS)

In [ ]:
#Вторая модель: голова на 75% разморожена, обучаем хвост и оставшуюся часть головы
model, base_model = train_model(model,base_model,0.25,LR*0.02,'2',
                                key_fit,train_datagen,test_datagen,EPOCHS)

***
***На следующем этапе количество эпох сокращено до 5, так как при большем количестве начинает падать метрика на валидационной выборке - переобучение.***
***

In [ ]:
#Третья модель: голова на 100% разморожена, обучаем хвост и оставшуюся часть головы
model, base_model = train_model(model,base_model,0.00,LR*0.01,'3',
                                key_fit,train_datagen,test_datagen,EPOCHS=5)

***
***Финальный график зависимости функции потерь и метрики приведен в конце notebook.***
***

# 7. Изменение IMAGE_SIZE, BATCH и AUGMENTATION

***
***Для повышения точности предсказания уменьшаем влияние аугментации и увеличиваем размер картинки, при этом снижаем размер батча, чтобы хватило памяти.***
***

In [ ]:
#Изменение глобальных настроек
EPOCHS         = 8
BATCH_SIZE     = 4
LR             = 1e-5
IMG_SIZE       = 512
IMG_CHANNELS   = 3
input_shape    = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)

***
***Количество эпох подобрано опытным путем. Оптимальным вариантом является 4-8.***
***

In [ ]:
p = 0.5 #Вероятность изменить изображение
#Создание аугментатора
augment = alb.Compose([
    #Поворот вокруг вертикальной оси
    alb.HorizontalFlip(p=p),
    #Вращение изображения
    alb.ShiftScaleRotate(shift_limit=0.0625,      #Коэффициент изменения сдвига
                         scale_limit=(0.1,0.2),   #Коэффициент изменение масштаба
                         interpolation=1,         #Флаг для вида интерполяции (линейная)
                         border_mode=4,           #Флаг для экстраполяцц (отражение)
                         rotate_limit=20,         #Угол поворота
                         p=0.7),                  #Вероятность
    #Изменение размера на заданный
    alb.Resize(IMG_SIZE, IMG_SIZE)
])
#Генерируем данные заного
#Генератор для обучающей и валидационной выборок
train_gen = ImageDataAugmentor(rescale=1.0/255,                     #Масштабирование
                               augment=augment,                     #Аугментатор
                               seed=RANDOM_SEED,                    #Для воспроизведения
                               validation_split=VAL_SPLIT)          #Размер выборки
#Генератор для тестовой выборки
test_gen = ImageDataGenerator(rescale=1.0/255)
#Создание обучающей выборки
train_datagen = train_gen.\
            flow_from_directory(PATH+'train/',                      #Путь
                                class_mode='categorical',           #Тип данных
                                batch_size=BATCH_SIZE,              #Размер BATCH'а
                                target_size=(IMG_SIZE, IMG_SIZE),   #Размер матрицы
                                shuffle=True,                       #Перемешиваем заранее
                                subset='training')                  #Название
#Создание валидационной выборки
test_datagen = train_gen.\
            flow_from_directory(PATH+'train/',                      #Путь
                                class_mode='categorical',           #Тип данных
                                batch_size=BATCH_SIZE,              #Размер BATCH'а
                                target_size=(IMG_SIZE, IMG_SIZE),   #Размер матрицы
                                shuffle=True,                       #Перемешиваем заранее
                                subset='validation')                #Название
#Создание тестовой выборки
test_sub_generator = test_gen.\
            flow_from_dataframe(dataframe=sample_submission,
                                x_col="Id",
                                y_col=None,
                                directory=PATH+'test_upload/',
                                class_mode=None,
                                batch_size=BATCH_SIZE,
                                shuffle=False,
                                target_size=(IMG_SIZE, IMG_SIZE),
                                seed=RANDOM_SEED)

In [ ]:
#Создаем голову заново для нового размера картинок
base_model = efn.EfficientNetB5(weights='imagenet',        #Обученная на imagenet
                                include_top=False,         #Включать ли верхнюю часть сети
                                input_shape=input_shape)   #Размер матрицы
#Загружаем веса с прошлой итерации
model.load_weights('best_model_3.hdf5')

In [ ]:
#Четвертая модель: все разморожено и изменения в BATCH и AUGMENTATION
model, base_model = train_model(model,base_model,0.00,LR,'4',
                                key_fit,train_datagen,test_datagen,EPOCHS)

***
***В результате качество на валидационной выборке было повышено на ~2%. Можно считать данную модель оптимальную из всех полученных ранее.***
***

# 8. Kaggle v.1

In [ ]:
def to_Kaggle(model):
    #Код взят с SkillFactory
    test_sub_generator.reset()
    predictions = model.predict_generator(test_sub_generator, steps=len(test_sub_generator), verbose=1) 
    predictions = np.argmax(predictions, axis=-1)
    label_map = (train_datagen.class_indices)
    label_map = dict((v,k) for k,v in label_map.items())
    predictions = [label_map[k] for k in predictions]
    filenames_with_dir=test_sub_generator.filenames
    submission = pd.DataFrame({'Id':filenames_with_dir, 'Category':predictions}, columns=['Id', 'Category'])
    submission['Id'] = submission['Id'].replace('test_upload/','')
    submission.to_csv('submission.csv', index=False)
    print('Save submit')
    pass

***
***Результат на Kaggle приведен в конце notebook.***
***

In [ ]:
#submit на Kaggle
to_Kaggle(model)

# 9.TTA

***
***Так как исходная модель была обучена на изображениях с аугментацией, то применим к тестовой выборке такую же аугментацию n раз. В результате усредним значения вероятностей.***
***

In [ ]:
p = 0.5 #Вероятность изменить изображение
#Создание аугментатора
augment = alb.Compose([
    #Замена каналов в RGB палитре
    alb.ChannelShuffle(p=p),
    #Размытие изображения
    alb.GaussianBlur(p=p),
    #Добавление шума
    alb.GaussNoise(p=p),
    #Поворот вокруг вертикальной оси
    alb.HorizontalFlip(p=p),
    #Изменение оттенка и насыщенности
    alb.HueSaturationValue(p=p),
    #Изменение яркости
    alb.RandomBrightness(p=p,limit=(0.2,0.4)),
    #Изменение контраста
    alb.RandomContrast(p=p,limit=(0.1,0.3)),
    #Сдвиг в RGB палитре
    alb.RGBShift(p=p),
    #Вращение изображения
    alb.ShiftScaleRotate(shift_limit=0.0625,      #Коэффициент изменения сдвига
                         scale_limit=(0.1,0.2),   #Коэффициент изменение масштаба
                         interpolation=1,         #Флаг для вида интерполяции (линейная)
                         border_mode=4,           #Флаг для экстраполяцц (отражение)
                         rotate_limit=20,         #Угол поворота
                         p=0.7),                  #Вероятность
    #Изменение размера на заданный
    alb.Resize(IMG_SIZE, IMG_SIZE)
])

In [ ]:
#Генерируем данные заного
#Генератор для обучающей и валидационной выборок
test_gen =  ImageDataAugmentor(rescale=1.0/255,                     #Масштабирование
                               augment=augment,                     #Аугментатор
                               seed=RANDOM_SEED,                    #Для воспроизведения
                               validation_split=VAL_SPLIT)          #Размер выборки
#Создание тестовой выборки
test_sub_generator = test_gen.\
            flow_from_dataframe(dataframe=sample_submission,
                                x_col="Id",
                                y_col=None,
                                directory=PATH+'test_upload/',
                                class_mode=None,
                                batch_size=BATCH_SIZE,
                                shuffle=False,
                                target_size=(IMG_SIZE, IMG_SIZE),
                                seed=RANDOM_SEED)

***
***Количество изменений было подобрано опытным путем. Значения 10-12 являются оптимальными. Больше не дает существенного прироста, а меньше уменьшают значение метрики.***
***

In [ ]:
#Количество изменений
n_tta = 12
#Массив для сохранений
predictions = []
#Перебор
for i in range(n_tta):
    print('{} / {}'.format(1+i, i))
    preds = model.predict(test_sub_generator, verbose=1) 
    predictions.append(preds)

***
***TTA позволил улучшить результат на ~0.2%.***
***

# 10. Kaggle v.2

***
***Результат на Kaggle приведен в конце notebook.***
***

In [ ]:
#Взятие среднего
pred = np.mean(predictions, axis=0)

In [ ]:
#Функция для вывода на Kaggle TTA
def to_Kaggle_TTA(predictions):
    predictions = np.argmax(predictions, axis=-1)
    label_map = (train_datagen.class_indices)
    label_map = dict((v,k) for k,v in label_map.items())
    predictions = [label_map[k] for k in predictions]
    filenames_with_dir=test_sub_generator.filenames
    submission = pd.DataFrame({'Id':filenames_with_dir, 'Category':predictions}, columns=['Id', 'Category'])
    submission['Id'] = submission['Id'].replace('test_upload/','')
    submission.to_csv('submission_TTA.csv', index=False)
    print('Save submit')
    pass

In [ ]:
#submit на Kaggle
to_Kaggle_TTA(pred)

***
# Вывод
***

In [ ]:
#Kaggle
#0.97453 - B5 + 16 + relu + TTA
#0.97438 - B5 + 16 + relu
#0.97348 - B5 + 32 + elu + TTA